<a href="https://colab.research.google.com/github/saiprem-eng/Brain-tumour-predictor/blob/master/brain_tumor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

In [ ]:
!kaggle datasets download -d sartajbhuvaji/brain-tumor-classification-mri

Dataset URL: https://www.kaggle.com/datasets/sartajbhuvaji/brain-tumor-classification-mri
License(s): MIT
 99% 86.0M/86.8M [00:06<00:00, 20.1MB/s]
100% 86.8M/86.8M [00:06<00:00, 14.7MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('brain-tumor-classification-mri.zip')
zip_ref.extractall()
zip_ref.close()

In [ ]:
train_dir="/content/Training"
test_dir="/content/Testing"

In [ ]:
# Setup data inputs
import tensorflow as tf
IMG_SIZE = (224, 224)
train_data_all = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                                label_mode="categorical",
                                                                                image_size=IMG_SIZE)

test_data_all= tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                label_mode="categorical",
                                                                image_size=IMG_SIZE,
                                                                shuffle=False) # don't shuffle test data for prediction analysis

Found 2870 files belonging to 4 classes.
Found 394 files belonging to 4 classes.


In [ ]:

labels = ['glioma_tumor','no_tumor','meningioma_tumor','pituitary_tumor']

In [ ]:


X_train = []
y_train = []
image_size = 150

# Training Data
for label in labels:
    folder_path = os.path.join('/content', 'Training', label)
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        img = plt.imread(img_path)
        img = tf.image.resize(img, (image_size, image_size))
        X_train.append(img)
        y_train.append(label)

# Testing Data (added to training data)
for label in labels:
    folder_path = os.path.join('/content', 'Testing', label)
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        img = plt.imread(img_path)
        img = tf.image.resize(img, (image_size, image_size))
        X_train.append(img)
        y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)


In [ ]:
from sklearn.utils import shuffle # Import the shuffle function

X_train, y_train = shuffle(X_train,y_train, random_state=101) # Shuffle the data

In [ ]:
from sklearn.model_selection import train_test_split # Import the train_test_split function

X_train,X_test,y_train,y_test = train_test_split(X_train,y_train, test_size=0.1,random_state=101) # Now you can use the function

In [ ]:
y_train_new = []
for i in y_train:
    y_train_new.append(labels.index(i))
y_train = y_train_new
y_train = tf.keras.utils.to_categorical(y_train)


y_test_new = []
for i in y_test:
    y_test_new.append(labels.index(i))
y_test = y_test_new
y_test = tf.keras.utils.to_categorical(y_test)

In [ ]:
image_size = 224  # Example image size
base_model = tf.keras.applications.efficientnet.EfficientNetB0(
    include_top=False,
    weights='imagenet',
    input_shape=(image_size, image_size, 3)
)
base_model.trainable = False

# Setup model architecture with trainable top layers
inputs = layers.Input(shape=(224, 224, 3), name="input_layer") # shape of input image
x = data_augmentation(inputs) # augment images (only happens during training)
x = base_model(x, training=False) # put the base model in inference mode so we can use it to extract features without updating the weights
x = layers.GlobalAveragePooling2D(name="global_average_pooling")(x)
x = layers.Dropout(rate=0.3)(x) # pool the outputs of the base model
outputs = layers.Dense(4, activation="softmax", name="output_layer")(x) # same number of outputs as classes
model_3 = tf.keras.Model(inputs, outputs)

# Compile
model_3.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(), # use Adam with default settings
              metrics=["accuracy"])

# Fit
history= model_3.fit(train_data_all,
                                           epochs=20, # fit for 5 epochs to keep experiments quick
                                           validation_data=test_data_all,
                                           validation_steps=int(len(test_data_all)), # evaluate on smaller portion of test data
                                           ) # save best model weights to file